In [1]:
import pandas as pd
df = pd.read_csv("spam.csv", encoding='latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [2]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df.rename(columns= {'v1': 'Category', 'v2': 'message'}, inplace= True)

In [25]:
df.groupby(['Category']).describe()

message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [3]:
df= pd.get_dummies(df, columns=['Category'])

In [28]:
df

,message,Category_ham,Category_spam
0,"Go until jurong point, crazy.. Available only ...",True,False
1,Ok lar... Joking wif u oni...,True,False
2,Free entry in 2 a wkly comp to win FA Cup fina...,False,True
3,U dun say so early hor... U c already then say...,True,False
4,"Nah I don't think he goes to usf, he lives aro...",True,False
...,...,...,...
5567,This is the 2nd time we have tried 2 contact u...,False,True
5568,Will Ì_ b going to esplanade fr home?,True,False
5569,"Pity, * was in mood for that. So...any other s...",True,False
5570,The guy did some bitching but I acted like i'd...,True,False


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.message, df.Category_spam, test_size = 0.25, random_state = 42)

We've taken the "Category_spam" column as y, so it will give True if the message is spam, False if not.

In [8]:
print(X_train.shape)

(4179,)


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
V = CountVectorizer()
X_train_count = V.fit_transform(X_train.values)
X_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [9]:
from sklearn.naive_bayes import MultinomialNB
model =  MultinomialNB()
model.fit(X_train_count, y_train)

MultinomialNB()

Testing 2 emails.

In [10]:
emails= [
    'Do not forget we have a meeting scheduled at 3 PM today in the main conference room.',#ham
    'Congratulations, You have Won a $1,000 Gift Card!'#spam
]
emails_count = V.transform(emails)
model.predict(emails_count)

array([False,  True])

Getting the accuracy

In [11]:
X_test_count = V.transform(X_test)
model.score(X_test_count, y_test)

0.9820531227566404

#### Using pipeline

In [15]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [16]:
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [17]:
clf.score(X_test, y_test)

0.9820531227566404